<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>


# Typeform - Log new Typeform entries in Notion databases


**Tags:** #typeform #notion #operations #automation


**Author:** [Sanjeet Attili](https://linkedin.com/in/sanjeet-attili-760bab190)

Capture and organize important project and customer details without slowing down for tedious copy-and-paste tasks! This template automatically adds each new entry received in Typeform as an item in your Notion database.
<br/>References :
- [TypeForm API](https://developer.typeform.com/responses/)
- [Notion API ](https://docs.naas.ai/drivers/notion)

Have a look at the sample typeform format used [here](https://1mx5hrd76qo.typeform.com/to/bw1oM4SP)

Sample output view of the notion database [here](https://billowy-lemming-95e.notion.site/f8e44ff261564c76b3bb80e6edb171a9?v=1d2a506563fe4082b71e78695185962e), which has all the questions asked in the typeform as column names and their responses as entries.

This output database consists of only 5 responses collected over the sample typeform.

## Input


### Import libraries


In [5]:
from naas_drivers import notion
from typeform import Typeform
import naas, pandas as pd
import requests
from datetime import datetime
import pydash as pd_, re

### Setup Typeform

- Get your [access token](https://developer.typeform.com/get-started/personal-access-token)


In [6]:
TYPEFORM_ACCESS_TOKEN = "<TYPEFORM_ACCESS_TOKEN>"

# Unique ID for the form. Find in your form URL.
# For example, in the URL "https://mysite.typeform.com/to/u6nXL7" the
# form_id is u6nXL7.

FORM_ID = "<FORM_ID>"

typeform = Typeform(TYPEFORM_ACCESS_TOKEN)

### Setup Notion

- Get your [Notion integration token ?](https://docs.naas.ai/drivers/notion)
- Share you database with your integration


In [7]:
NOTION_TOKEN = "<NOTION_TOKEN>"
# database-url
# https://www.notion.so/naas-official/f89fddc31128400fab11001a215aff09?v=d84b89b704c7dssd432350cc273
DATABASE_URL = "<DATABASE_URL>"

### Setup Naas


In [ ]:
# Schedule your notebook everyday at 9 AM
naas.scheduler.add(cron="0 9 * * *")

#-> Uncomment the line below to remove your scheduler
# naas.scheduler.delete()


## Model


### Get data from Typeform

- Currently supported data types for retreiving data from typeform with this notebook are as follows:
  - Text (short and long)
  - Number
  - Rating
  - Multi-choice with one or more correct responses
  - Phone number
  - Email

**Collecting questions from typeform to setup as column names in database**

In [8]:
def get_questions_dict():
    fields = typeform.forms.get(FORM_ID)['fields']
    questions={}
    for field in fields:
        new_str = re.sub('{{field:' + r'[0-9A-Z]+'+'}}, ', '', field['title'])
        questions[field['id']] = new_str
    return questions

questions = get_questions_dict()
questions

{'8wQbkuiT2P0c': "Hello, what's your name?",
 'SwMAxJlR3i7N': 'Nice to meet you, how is your day going?',
 'WPChqR9aJLhj': 'How old are you?',
 'e8oxwTuqL0Pi': 'Enter your email address',
 '1ZZ68d6UsPcP': 'Give us your contact details!',
 'm349WCUgaIpa': 'Select options which are true',
 'SM5ohn9x03ee': 'Please give us your rating',
 'zPxehRIt67Yq': 'Please enter your feedback on your previous purchase with us!'}

**Collecting answers from typeform**

In [9]:
def get_answers(form_id, token):
    url = f"https://api.typeform.com/forms/{form_id}/responses"
    headers = {'Authorization': f'Bearer {token}'}

    r = requests.get(url, headers = headers)
    responses = pd_.get(r.json(), 'items')
    answers= []
    for resp in responses:
        response_answer =[]
        for field in pd_.get(resp, 'answers'):
            entries={}
            entries['id'], entries['type'] = pd_.get(field, 'field.id'), pd_.get(field, 'type')
    
            if entries['type'] == 'text':
                entries['answer'] = pd_.get(field, f"{entries['type']}")
                
            elif entries['type'] == 'choice':
                entries['answer'] = pd_.get(field, 'choice.label')
                
            elif entries['type'] == 'choices':
                entries['answer'] = pd_.get(field, 'choices.labels')
                
            elif entries['type'] == 'number':
                entries['answer'] = pd_.get(field, 'number')
                
            elif entries['type'] == 'phone_number':
                entries['answer'] = pd_.get(field, 'phone_number')
            
            elif entries['type'] == 'email':
                entries['answer'] = pd_.get(field, 'email')
                
            response_answer.append(entries)
        answers.append(response_answer)
    return answers

answers = get_answers(FORM_ID, TYPEFORM_ACCESS_TOKEN)
answers[0]

[{'id': '8wQbkuiT2P0c', 'type': 'text', 'answer': 'he-man'},
 {'id': 'SwMAxJlR3i7N', 'type': 'choice', 'answer': 'Not so well...'},
 {'id': 'WPChqR9aJLhj', 'type': 'number', 'answer': 15},
 {'id': 'e8oxwTuqL0Pi', 'type': 'email', 'answer': 'xyz@gmail.com'},
 {'id': '1ZZ68d6UsPcP', 'type': 'phone_number', 'answer': '+918732413212'},
 {'id': 'm349WCUgaIpa', 'type': 'choices', 'answer': ['True']},
 {'id': 'SM5ohn9x03ee', 'type': 'number', 'answer': 5},
 {'id': 'zPxehRIt67Yq', 'type': 'text', 'answer': 'superb product'}]

In [10]:
def get_typeform_data(form_id, token, questions, answers):
    # Get response dataframe
    # Retrieve response and map field id with field title to column name's
    df = pd.DataFrame()
    url = f"https://api.typeform.com/forms/{form_id}/responses"
    headers = {'Authorization': f'Bearer {token}'}

    r = requests.get(url, headers = headers)
    lst_responses = pd_.get(r.json(), 'items')
    for idx, resp in enumerate(lst_responses):
        in_time, out_time = pd_.get(resp, 'landed_at'), pd_.get(resp, 'submitted_at')
        time_diff = datetime.strptime(out_time, '%Y-%m-%dT%H:%M:%SZ') - datetime.strptime(in_time, '%Y-%m-%dT%H:%M:%SZ')
        if str(time_diff).split(':')[1] == '00':
            df.loc[idx, 'time_taken_to_fill_form'] = ":".join(str(time_diff).split(':')[1:]) + 'secs'
        else:
            df.loc[idx, 'time_taken_to_fill_form'] = ":".join(str(time_diff).split(':')[1:]) + 'mins'
        
        df.loc[idx, 'response_id'] = pd_.get(resp, 'response_id')
    
    for idx, response in enumerate(answers):
        for entity in response:
            if entity['type'] == 'choices':
                df.loc[idx, questions[entity['id']]] = ",".join(entity['answer'])
            else:
                df.loc[idx, questions[entity['id']]] = entity['answer']
                
    df.fillna('None', inplace=True)
    
    return df

df_typeform = get_typeform_data(FORM_ID, TYPEFORM_ACCESS_TOKEN, questions, answers)
df_typeform

,time_taken_to_fill_form,response_id,"Hello, what's your name?","Nice to meet you, how is your day going?",How old are you?,Enter your email address,Give us your contact details!,Select options which are true,Please give us your rating,Please enter your feedback on your previous purchase with us!
0,00:58secs,dledwaka744lsadleyqd5xym7z4gcx2b,he-man,Not so well...,15.0,xyz@gmail.com,+918732413212,True,5.0,superb product
1,00:50secs,6r9h3epkfg9cgbik5n6r9h77hh1r8bjg,Attili Sanjeet,Not so well...,28.0,attili18100@iiitnr.edu.in,+919384592191,"True,True",3.0,service is not that good
2,01:08mins,dc02pdr56u1hfrsvrj3dc02pd9xjhxld,rakesh kumar saami,Terrific!,37.0,None,+919342193029,"True,True,True",4.0,"Good service, but to be improved more"
3,00:26secs,7x619d73vsmbxionl58427x6105ymx96,Rahul,Not so well...,None,None,None,None,None,worst product i can ever ask for
4,00:21secs,joq4pglxr0b0o7d28t5jjjoq4pgqiadi,Sanju,Terrific!,None,None,None,None,None,Made a good purchase after a long time!


### Get data from Notion DB


In [24]:
pages = notion.connect(NOTION_TOKEN).database.query(DATABASE_URL, query={})
len(pages)

6

### Adding data to Notion


In [25]:
def add_new_entries(df):
    if df.shape[0]==0:
        return df
    
    columns = df.columns.to_list()
    for col in columns:
        if 'name' in col:
            name_col = col
    
    columns.remove(name_col)
    
    for idx, row in df.iterrows():
        if idx == df.shape[0]:
            break
        page = notion.connect(NOTION_TOKEN).page.create(database_id=DATABASE_URL, title= row[name_col])
        for column in columns:
            page.rich_text(column, str(row[column]))
            page.update()
    return df

In [26]:
def add_data_to_notion(df_typeform, pages):
    id_present = False
    try:
        pages[0].properties['response_id']
        id_present = True
    except KeyError:
        pass
    
    # If no data is present initially
    if not id_present:
        df = add_new_entries(df_typeform)
    
    # If some data exists
    else:
        notion_df = notion.connect(NOTION_TOKEN).database.get(DATABASE_URL).df()
        existing_ids = notion_df.response_id.to_list()
        
        new_entries = df_typeform[df_typeform.response_id.isin(existing_ids) == False]
        df = add_new_entries(new_entries)
    
    return df

df_notion = add_data_to_notion(df_typeform, pages)

True


## Output


In [27]:
df_notion.head()

,time_taken_to_fill_form,response_id,"Hello, what's your name?","Nice to meet you, how is your day going?",How old are you?,Enter your email address,Give us your contact details!,Select options which are true,Please give us your rating,Please enter your feedback on your previous purchase with us!
